In [1]:
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/briandepasquale/Documents/GitHub/CoSyNe_2025_GNN_tutorial/.venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/briandepasquale/Documents/GitHub/CoSyNe_2025_GNN_tutorial/.venv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/briandepasquale/Documents/GitHub/CoSy

In [2]:

# ----------------------------------------------------------------------
# 1) Create GDC transform
# ----------------------------------------------------------------------
gdc = T.GDC(
    self_loop_weight=1,
    normalization_in='sym',
    normalization_out='col',
    diffusion_kwargs=dict(method='ppr', alpha=0.05),
    sparsification_kwargs=dict(method='topk', k=128, dim=0),
    exact=False,
)


In [4]:

# ----------------------------------------------------------------------
# 2) Load dataset and apply GDC
# ----------------------------------------------------------------------
#dataset = Planetoid(root='data/Planetoid', name='Cora', transform=gdc)
dataset = Planetoid(root='data/Planetoid', name='Cora')
data = dataset[0]

# Now data.edge_index and data.edge_attr (if any) represent the diffused graph


FileNotFoundError: https://github.com/kimiyoung/planetoid/raw/master/data/ind.cora.x

In [ ]:
# data is a torch_geometric.data.Data object
data = gdc(data)

In [ ]:

# ----------------------------------------------------------------------
# 3) Define a simple GCN to use on the new diffused graph
# ----------------------------------------------------------------------
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GCN(dataset.num_features, 64, dataset.num_classes).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)


In [ ]:

# ----------------------------------------------------------------------
# 4) Training loop
# ----------------------------------------------------------------------
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return float(loss)

def test():
    model.eval()
    logits = model(data.x, data.edge_index)
    preds = logits.argmax(dim=1)
    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        acc = (preds[mask] == data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

for epoch in range(1, 201):
    loss = train()
    train_acc, val_acc, test_acc = test()
    if epoch % 20 == 0:
        print(f"Epoch: {epoch:03d}, Loss: {loss:.4f}, "
              f"Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}")
d